In [1]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 225kB 11.6MB/s 
     |████████████████████████████████| 112kB 21.7MB/s 
     |████████████████████████████████| 245kB 10.6MB/s 
     |████████████████████████████████| 2.1MB 11.2MB/s 
     |████████████████████████████████| 3.3MB 55.4MB/s 
     |████████████████████████████████| 901kB 58.1MB/s 


In [3]:
from datasets import load_dataset
dataset = load_dataset('squad_kor_v1')

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/92f88eedc7d67b3f38389e8682eabe68caa450442cc4f7370a27873dbc045fe4. Subsequent calls will reuse this data.


In [48]:
corpus = list(set([example['context'] for example in dataset['train']]))

In [49]:
len(corpus)

9606

In [6]:
corpus[0]

'모기는 사람, 가축에게 피를 빨아먹으며 전염병을 옮긴다. 또한 물리면 가려운데, 이것은 모기가 피가 응고되지 말라고 내보내는 성분 때문이다. 긁거나 침을 바르거나 하면 세균감염의 위험이 있다. 모기가 매개하는 질병은 뇌염·말라리아(학질)·필라리아(상피병)·일본뇌염·황열병 등이며 전 세계에 약 3,000종 가량이 알려져 있다. 원래 모기는 애벌레 시기에 천적인 큰 물고기, 자라, 물방개, 가물치, 송사리, 미꾸라지, 금붕어에 의해 수가 조절되지만, 각종 개발과 공업화로 생태계가 파괴되면서 상대적으로 천적이 사라진 모기는 비정상적으로 늘어났고, 매년 그 수가 급증하였다. 지구 온난화와 환경 오염으로 모기가 늘어나고 있다.'

In [9]:
tokenizer_func = lambda x : x.split(' ')

In [10]:
tokenizer_func(corpus[0])[:10]

['모기는', '사람,', '가축에게', '피를', '빨아먹으며', '전염병을', '옮긴다.', '또한', '물리면', '가려운데,']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vertorizer = TfidfVectorizer(tokenizer= tokenizer_func, ngram_range =(1,2))

In [12]:
vertorizer.fit(corpus)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function <lambda> at 0x7f101b0234d0>, use_idf=True,
                vocabulary=None)

In [13]:
sp_matrix = vertorizer.transform(corpus)

In [15]:
sp_matrix.shape #n=1,2 모두 포함되 결과 -> 1272768

(9606, 1272768)

In [24]:
import pandas as pd
# print(type(sp_matrix[0].T)) 
# <class 'scipy.sparse.csc.csc_matrix'>
# print(type(sp_matrix[0].T.todense()))
# <class 'numpy.matrix'>
# todense() -> sparse matrix 가 numpy.ndarray 로 변환
df = pd.DataFrame(sp_matrix[0].T.todense(), index = vertorizer.get_feature_names(), columns = ['TF-IDF'])
df = df.sort_values('TF-IDF', ascending = False)
print(df.head(10))

             TF-IDF
모기가        0.257690
모기는        0.246665
수가         0.090529
개발과 공업화로   0.085897
긁거나 침을     0.085897
가물치, 송사리,  0.085897
수가 조절되지만,  0.085897
성분 때문이다.   0.085897
각종 개발과     0.085897
옮긴다. 또한    0.085897


In [53]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

In [27]:
query_vec = vertorizer.transform([query])

In [28]:
query_vec.shape

(1, 1272768)

In [41]:
result = query_vec * sp_matrix.T
result.shape
result

<1x9606 sparse matrix of type '<class 'numpy.float64'>'
	with 361 stored elements in Compressed Sparse Row format>

In [42]:
# https://rfriend.tistory.com/551
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [43]:
k = 3
doc_scores[:k],doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([3907, 2130, 4124], dtype=int32))

In [45]:
print('[Serch query]\n', query, '\n')
print('[Ground truth passage]')
print(ground_truth,"\n")

for i in range(k):
  print('[Top-%d passage with score %.4f'%(i+1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id],'\n')

[Serch query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

[Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

[Top-2